In [4]:
import azureml
from azureml.core import Workspace
workspaces = Workspace.list("3165a1c1-fd45-4c8d-938e-0058c823f960")
ws = workspaces["aml-playground"]
playground_ws = ws[0]
playground_ws

Workspace.create(name='aml-playground', subscription_id='3165a1c1-fd45-4c8d-938e-0058c823f960', resource_group='aml-playground')

In [5]:
import mlflow

## Construct AzureML MLFLOW TRACKING URI
def get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace):
    return "azureml://{}.api.azureml.ms/mlflow/v1.0/subscriptions/{}/resourceGroups/{}/providers/Microsoft.MachineLearningServices/workspaces/{}".format(region, subscription_id, resource_group, workspace)

region='northeurope' ## example: westus
subscription_id = '3165a1c1-fd45-4c8d-938e-0058c823f960' ## example: 11111111-1111-1111-1111-111111111111
resource_group = 'aml-playground' ## example: myresourcegroup
workspace = 'aml-playground' ## example: myworkspacename

MLFLOW_TRACKING_URI = get_azureml_mlflow_tracking_uri(region, subscription_id, resource_group, workspace)

## Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

## Make sure the MLflow URI looks something like this: 
## azureml://<REGION>.api.azureml.ms/mlflow/v1.0/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.MachineLearningServices/workspaces/<AML_WORKSPACE_NAME>

print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)

MLFlow Tracking URI: azureml://northeurope.api.azureml.ms/mlflow/v1.0/subscriptions/3165a1c1-fd45-4c8d-938e-0058c823f960/resourceGroups/aml-playground/providers/Microsoft.MachineLearningServices/workspaces/aml-playground


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException, LocalTarget

# Choose a name for your cluster.
cluster_name = "preproject-cluster"
# instance_name = "cpuinstance1"
# cluster_name = instance_name


run_locally = False

if run_locally:
    compute_target = LocalTarget()
else:
    # Run in cloud

    # Check if we find a cluster with the name
    try:
        compute_target = ComputeTarget(workspace=playground_ws, name=cluster_name)
        print('Found existing compute target.')
    except ComputeTargetException:
        # Make a new one if an existing target is not found.
        print('Creating a new compute target...')
        compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_F4s_v2',
                                                            max_nodes=1,
                                                            idle_seconds_before_scaledown=300)

        # Create the cluster.
        compute_target = ComputeTarget.create(playground_ws, cluster_name, compute_config)

        compute_target.wait_for_completion(show_output=True)

# Use get_status() to get a detailed status for the current AmlCompute.
if not isinstance(compute_target, LocalTarget):
    print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-05-06T09:33:07.069000+00:00', 'errors': None, 'creationTime': '2022-05-05T17:16:06.630081+00:00', 'modifiedTime': '2022-05-05T17:16:10.173469+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT300S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_F4S_V2'}


In [7]:
from azureml.core import ScriptRunConfig
from azureml.core.environment import Environment as AmlEnvironment
from azureml.core import Experiment


auv_docker_env = AmlEnvironment.from_docker_image("preproject", "krisbrud.azurecr.io/preproject")
# experiment_name = ""
config_name = "sippo-weighted-path-follow2"

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=["--config", config_name], # , "--timesteps", int(1e6)], #, "--timesteps", int(1e6)],
                      compute_target=compute_target,
                      environment=auv_docker_env)
run = Experiment(playground_ws, config_name).submit(src)
# run.wait_for_completion(show_output=True)
# Note: If running a 
